In [1]:
import json
import typing

import pandas as pd
import requests
import tqdm

import config

In [2]:
CFG = config.Config()

In [3]:
MODEL: str = "llama3:70b-instruct-q6_K"

TASK_NAME: str = "political_ideology"
TASK_TEMPLATE: str = \
"""
Classify the following message as ideologically liberal (0), ideologically neutral (1), or ideologically conservative (2). Ideology here is defined in the context of the German political system. Messages with no ideological content are classified as neutral. 

Respond with only the predicted class (0 or 1 or 2) of the request.

Text: {text}
Class:
"""

In [4]:
user_content: pd.DataFrame = pd.read_parquet(CFG.final_files["user_content"]).reset_index("userId")
user_content

,userId,content,createdAt
id,,,
66252255616a4cb7061ee32c,661d1639b8beabb58229451b,#UkraineKrieg: Beunruhigende Entwicklungen in ...,2024-04-21 14:27:33.315
66253e74616a4cb7061ee7cc,661d1639b8beabb58229451b,"Bedauerlich, dass gewaltsame Konflikte weiterh...",2024-04-21 16:27:32.919
66255134616a4cb7061eeaf4,661d1639b8beabb58229451b,Gewalt im Nahen Osten & Ukraine nicht isoliert...,2024-04-21 17:47:32.762
6627d8efa6b56b19b4300ae4,661d1639b8beabb58229451b,"Unterstützen wir unsere Verbündeten, respektie...",2024-04-23 15:51:11.067
6628b08fa6b56b19b4301867,661d1639b8beabb58229451b,Geschlossene Tür für Unsicherheit: US-Repräsen...,2024-04-24 07:11:11.797
...,...,...,...
662a070537d6395f42ca7bfd,662a070537d6395f42ca7bf9,Kleiner vermieter Junge noch nicht gefunden,2024-04-25 07:32:21.584
662a12fa37d6395f42ca87fe,662a12fa37d6395f42ca87fa,Lok Leipzig holt Jochen Seitz als Trainer,2024-04-25 08:23:22.984
662a30f137d6395f42ca99c6,662a30f137d6395f42ca99c2,Russland droht der USA,2024-04-25 10:31:13.258


In [6]:
predictions: typing.Dict[str, str] = {}

In [7]:
for index, row in tqdm.tqdm(user_content.iterrows()):
    try:
        predictions[index] = requests.post(
            'https://inf.cl.uni-trier.de/',
            json={
                'model': MODEL,
                'system': TASK_TEMPLATE,
                'prompt': TASK_TEMPLATE.format(text=row.content)
            }).json()['response'].strip()

    except json.JSONDecodeError:
        print("Invalid JSON response, skipping to next sample.")


592it [06:30,  1.52it/s]


In [8]:
predictions

{'66252255616a4cb7061ee32c': '1',
 '66253e74616a4cb7061ee7cc': '0',
 '66255134616a4cb7061eeaf4': '0',
 '6627d8efa6b56b19b4300ae4': '1',
 '6628b08fa6b56b19b4301867': '0',
 '6628c354a6b56b19b4301c88': '1',
 '6629361a889820447de11833': '0',
 '66293acc889820447de11933': '0',
 '6629442b889820447de11b4e': '2',
 '662a207b889820447de12a6a': '1',
 '662a252a889820447de12b86': '0',
 '662b7b5b889820447de15738': '0',
 '662bb84a889820447de1653b': '0',
 '662bebdb889820447de17133': '0',
 '662e0b9b889820447de1a757': '0',
 '662e8a2b889820447de1c457': '1',
 '662fc43c889820447de1e926': '1',
 '6624c494616a4cb7061ed367': '0',
 '6624e0b3616a4cb7061ed81a': '0',
 '66253065616a4cb7061ee57a': '0',
 '662555e5616a4cb7061eebc3': '1',
 '662670e7d4ec55a25b930f20': '0',
 '662772b8d4ec55a25b931e16': '1',
 '6628b53fa6b56b19b4301964': '2',
 '6628be9fa6b56b19b4301b8b': '1',
 '6629523a889820447de11e97': '0',
 '66295b99889820447de120ab': '2',
 '662a045b889820447de123f0': '0',
 '662a58bd889820447de13734': '0',
 '662d1c8b8898

In [9]:
predicted: pd.DataFrame = user_content.join(pd.Series(predictions, name=TASK_NAME))
predicted["political_ideology"].value_counts()

political_ideology
0    276
1    274
2     42
Name: count, dtype: int64

In [10]:
predicted[predicted["political_ideology"] == "2"]

,userId,content,createdAt,political_ideology
id,,,,
6629442b889820447de11b4e,661d1639b8beabb58229451b,"Freut mich zu hören, dass der US-Kongress die ...",2024-04-24 17:40:59.206,2
6628b53fa6b56b19b4301964,661d163cb8beabb58229451c,"Stolz darauf, dass das Repräsentantenhaus für ...",2024-04-24 07:31:11.646,2
66295b99889820447de120ab,661d163cb8beabb58229451c,Stärkung von Verbündeten bedeutet nicht Einmis...,2024-04-24 19:20:57.963,2
6627dd9fa6b56b19b4300be5,661d1641b8beabb58229451f,Unterstützen wir unsere ukrainischen Verbündet...,2024-04-23 16:11:11.485,2
6629235b889820447de113d2,661d1641b8beabb58229451f,Schweigende Hände sind golden: Russlands Bekäm...,2024-04-24 15:20:59.403,2
662a126b889820447de1272a,661d1641b8beabb58229451f,Unterstützen wir souveräne Nationen wie die Uk...,2024-04-25 08:20:59.202,2
662df8da889820447de1a2e6,661d1641b8beabb58229451f,Starke Wirtschaft dank begrenzter Regierung & ...,2024-04-28 07:20:58.740,2
662fb62c889820447de1e5f1,661d1646b8beabb582294522,Interessante Einschätzung. Dennoch warnt ein K...,2024-04-29 15:01:00.754,2
6630c60b889820447de200fe,661d1646b8beabb582294522,Handelsblatt - verlässliche Nachrichten für ko...,2024-04-30 10:20:59.863,2
